In [13]:
import psycopg2
from dotenv import load_dotenv
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

import warnings
warnings.filterwarnings('ignore')

In [5]:
CURR_DIR=os.getcwd()
print(CURR_DIR)
env_file_path=os.path.join(os.path.dirname(CURR_DIR), '.env')
print(env_file_path)

# load the environment variables
load_dotenv(env_file_path)
env=os.environ

DBNAME=env.get('DBNAME')
USERNAME=env.get('USERNAME')
PASSWORD=env.get('PASSWORD')
HOSTNAME=env.get('HOSTNAME')
PORT=env.get('PORT')
SCHEMA=env.get('SCHEMA')
print(f"Schema : {SCHEMA}")
print(f"DBName: {DBNAME} HOSTNAME: {HOSTNAME}")

BASE_DIR=os.path.dirname(os.getcwd())
print(BASE_DIR)

DATA_DIR=os.path.join(BASE_DIR,'data')
print(DATA_DIR)

/Users/dare_devil/Desktop/MLDS_2024/Quarter1/Everything_Starts_With_Data/Project/MLDS_400_G11/code
/Users/dare_devil/Desktop/MLDS_2024/Quarter1/Everything_Starts_With_Data/Project/MLDS_400_G11/.env
Schema : group_11
DBName: everything2023 HOSTNAME: pg.analytics.northwestern.edu
/Users/dare_devil/Desktop/MLDS_2024/Quarter1/Everything_Starts_With_Data/Project/MLDS_400_G11
/Users/dare_devil/Desktop/MLDS_2024/Quarter1/Everything_Starts_With_Data/Project/MLDS_400_G11/data


In [6]:
# Establish the connection
conn = psycopg2.connect(
    dbname=DBNAME,
    user=USERNAME,
    password=PASSWORD,
    host=HOSTNAME,
    port=PORT
)

# Create a cursor object to interact with the database
cur = conn.cursor()

# Setting the SearchPath to Group Schema
cur.execute(f"SET search_path TO {SCHEMA};")

In [7]:
# Getting Data from DeptInfo
cur.execute("SELECT * FROM deptinfo;")
column_names = [desc[0] for desc in cur.description]
print(column_names)

rows = cur.fetchall()
df_deptinfo = pd.DataFrame(rows, columns=column_names)
df_deptinfo.head()

['DEPT', 'DEPTDESC']


,DEPT,DEPTDESC
0,800,CLINIQUE
1,801,LESLIE
2,1100,GARY F
3,1107,JACQUES
4,1202,CABERN


In [8]:
# Getting Data from Skstinfo
cur.execute("SELECT * FROM skstinfo;")
column_names = [desc[0] for desc in cur.description]
print(column_names)

rows = cur.fetchall()
df_skstinfo = pd.DataFrame(rows, columns=column_names)
df_skstinfo.head()

['SKU', 'STORE', 'COST', 'RETAIL']


,SKU,STORE,COST,RETAIL
0,3,102,123.36,440.0
1,3,103,123.36,440.0
2,3,104,123.36,440.0
3,3,202,123.36,440.0
4,3,203,123.36,440.0


In [9]:
# Getting Data from skutinfo
cur.execute("SELECT * FROM skuinfo;")
column_names = [desc[0] for desc in cur.description]
print(column_names)

rows = cur.fetchall()
df_skuinfo = pd.DataFrame(rows, columns=column_names)


df_skuinfo_orig = df_skuinfo.copy()
df_skuinfo.head()

['SKU', 'DEPT', 'CLASSID', 'UPC', 'STYLE', 'COLOR', 'SIZE', 'PACKSIZE', 'VENDOR', 'BRAND']


,SKU,DEPT,CLASSID,UPC,STYLE,COLOR,SIZE,PACKSIZE,VENDOR,BRAND
0,3,6505,113,400000003000,00 F55KT2,WHISPERWHITE,P8EA,1,5119207,TURNBURY
1,4,8101,002,400000004000,22 615CZ4,SPEARMI,S,1,3311144,C A SPOR
2,5,7307,003,400000005000,7LBS 245-01,34 SILVER,KING,1,5510554,BEAU IDE
3,8,3404,00B,400000008000,622 F05H84,MORNING MI,2T,1,2912827,HARTSTRI
4,15,2301,004,400000015000,126 MDU461,255CAMEL,12,1,23272,JONES/LA


In [10]:
# Getting Data from strinfo
cur.execute("SELECT * FROM strinfo;")
column_names = [desc[0] for desc in cur.description]
print(column_names)

rows = cur.fetchall()
df_strinfo = pd.DataFrame(rows, columns=column_names)
df_strinfo.head()

['STORE', 'CITY', 'STATE', 'ZIP']


,STORE,CITY,STATE,ZIP
0,2,ST. PETERSBURG,FL,33710
1,3,ST. LOUIS,MO,63126
2,4,LITTLE ROCK,AR,72201
3,7,FORT WORTH,TX,76137
4,9,TEMPE,AZ,85281


In [11]:
#Construct store-level features
# state_demographics.csv : open source state level data
df_demo= pd.read_csv(os.path.join(DATA_DIR,'state_demographics.csv'))
df_demo.head()

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,...,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,...,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,...,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,...,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,...,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9


In [16]:
state_mapping = json.load(open(os.path.join(DATA_DIR,'state_mapping.json')))
print(state_mapping)

{'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


In [17]:
df_demo['STATE'] = df_demo['State'].map(state_mapping)

# Calculating Men and Women Ratio in the state
df_demo['MenRatio'] = df_demo['Men'] / df_demo['TotalPop']
df_demo['WomenRatio'] = df_demo['Women'] / df_demo['TotalPop']

In [18]:
df_selected=df_demo[['STATE','MenRatio','WomenRatio','White','Black','Native','Asian','Pacific','Income','Poverty','Professional','Service','Office','Construction','Production','Unemployment']]
result_df = df_selected.groupby('STATE').mean().reset_index()

In [65]:
trans_data_path = os.path.join(DATA_DIR,'transact_grp_sku_store_quarter_v1.csv')
df_trans = pd.read_csv(trans_data_path)
df_trans.head()

,SKU,STORE,quarter,totalunitssold,averageoriginalprice,averagesellingprice,purchase_count,return_count,purchase_prob
0,3,2007,Q4,2,0.00,4.50,2,0,1.0
1,4,7602,Q4,2,0.00,40.00,2,0,1.0
2,5,2503,Q1,2,0.00,50.00,2,0,1.0
3,24,2204,Q3,2,21.99,21.99,2,0,1.0
4,55,1607,Q3,2,69.00,34.50,2,0,1.0


In [66]:
def find_empty_spaces(df, col_name):
    print(f"Analysing {col_name}")
    for _ in df[col_name].unique():
        if not(len(_) == len(_.strip()) and len(_) > 0):
            print(f"Found Empty Spaces! Fix {col_name}")
            break
            
def _fix_empty_spaces(entry):
    if len(entry) == 0:
        return None
    
    return entry.strip()

def fix_empty_spaces(df, col_name):
    df[col_name] = df[col_name].apply(lambda x : _fix_empty_spaces(x))
    df.dropna(subset=[col_name], inplace=True)
    return df

In [67]:
find_empty_spaces(df=df_deptinfo, col_name='DEPTDESC')

Analysing DEPTDESC


In [68]:
find_empty_spaces(df_skuinfo, col_name="STYLE")
find_empty_spaces(df_skuinfo, col_name="COLOR")
find_empty_spaces(df_skuinfo, col_name="SIZE")
find_empty_spaces(df_skuinfo, col_name="BRAND")

df_skuinfo = fix_empty_spaces(df_skuinfo, col_name="COLOR")
find_empty_spaces(df_skuinfo, col_name="COLOR")
df_skuinfo = fix_empty_spaces(df_skuinfo, col_name="BRAND")
find_empty_spaces(df_skuinfo, col_name="BRAND")

Analysing STYLE
Analysing COLOR
Analysing SIZE
Analysing BRAND
Analysing COLOR
Analysing BRAND


In [69]:
find_empty_spaces(df_strinfo, 'CITY')
find_empty_spaces(df_strinfo, 'STATE')

fix_empty_spaces(df_strinfo, 'CITY')
fix_empty_spaces(df_strinfo, 'STATE')

find_empty_spaces(df_strinfo, 'CITY')
find_empty_spaces(df_strinfo, 'STATE')

Analysing CITY
Analysing STATE
Analysing CITY
Analysing STATE


In [70]:
df_sku = df_skuinfo.copy()
df_str = df_strinfo.copy()
df_skst = df_skstinfo.copy()
#df_trans = df_trnsact.copy()

print(df_sku.shape[0], df_str.shape[0], df_skst.shape[0])

1531682 453 20640000


In [71]:
df_sku.head()

,SKU,DEPT,CLASSID,UPC,STYLE,COLOR,SIZE,PACKSIZE,VENDOR,BRAND
0,3,6505,113,400000003000,00 F55KT2,WHISPERWHITE,P8EA,1,5119207,TURNBURY
1,4,8101,002,400000004000,22 615CZ4,SPEARMI,S,1,3311144,C A SPOR
2,5,7307,003,400000005000,7LBS 245-01,34 SILVER,KING,1,5510554,BEAU IDE
3,8,3404,00B,400000008000,622 F05H84,MORNING MI,2T,1,2912827,HARTSTRI
4,15,2301,004,400000015000,126 MDU461,255CAMEL,12,1,23272,JONES/LA


In [72]:
df_sku = df_sku[['SKU','DEPT', 'PACKSIZE', 'VENDOR', 'BRAND', 'CLASSID']]
df_skst = df_skst[['SKU','STORE','COST','RETAIL']]

In [73]:
df_str_skst = df_skst.merge(df_str, on='STORE', how='inner')
df_str_skst.head()

,SKU,STORE,COST,RETAIL,CITY,STATE,ZIP
0,3,102,123.36,440.0,TAMPA,FL,33607
1,15,102,47.60,119.0,TAMPA,FL,33607
2,45,102,20.66,59.0,TAMPA,FL,33607
3,180,102,11.25,25.0,TAMPA,FL,33607
4,210,102,42.50,85.0,TAMPA,FL,33607


In [74]:
df_sku_skst = df_sku.merge(df_str_skst, on='SKU', how='inner')
print(df_sku_skst.shape[0], df_sku.shape[0], df_str_skst.shape[0])
df_sku_skst.head()

20522505 1531682 20640000


,SKU,DEPT,PACKSIZE,VENDOR,BRAND,CLASSID,STORE,COST,RETAIL,CITY,STATE,ZIP
0,3,6505,1,5119207,TURNBURY,113,102,123.36,440.0,TAMPA,FL,33607
1,3,6505,1,5119207,TURNBURY,113,103,123.36,440.0,ST LOUIS,MO,63117
2,3,6505,1,5119207,TURNBURY,113,104,123.36,440.0,CHARLOTTE,NC,28216
3,3,6505,1,5119207,TURNBURY,113,202,123.36,440.0,TAMPA,FL,33612
4,3,6505,1,5119207,TURNBURY,113,203,123.36,440.0,CHESTERFIELD,MO,63017


In [75]:
df_sku_skst_trans = df_sku_skst.merge(df_trans, on=['SKU','STORE'], how='inner')
print(df_sku_skst_trans.shape[0], df_sku_skst.shape[0], df_trans.shape[0])
df_sku_skst_trans.head()

6042459 20522505 21112820


,SKU,DEPT,PACKSIZE,VENDOR,BRAND,CLASSID,STORE,COST,RETAIL,CITY,STATE,ZIP,quarter,totalunitssold,averageoriginalprice,averagesellingprice,purchase_count,return_count,purchase_prob
0,3,6505,1,5119207,TURNBURY,113,2007,123.36,440.00,SAN ANTONIO,TX,78216,Q4,2,0.00,4.500000,2,0,1.000000
1,69,7106,1,4816434,ME TOO,915,8002,22.00,12.25,BATON ROUGE,LA,70815,Q3,3,12.25,11.460000,3,0,1.000000
2,78,5301,1,9912767,M.M. & R,001,4202,24.30,40.20,PENSACOLA,FL,32504,Q3,4,60.00,42.135000,2,2,0.500000
3,91,9801,1,9212766,GABAR IN,004,3303,15.00,17.00,HUTCHINSON,KS,67501,Q3,3,68.00,38.533333,1,2,0.333333
4,123,3701,1,5016174,DANNY &,435,3102,25.31,49.99,JACKSONVILLE,FL,32225,Q3,3,80.00,34.990002,2,1,0.666667


In [76]:
df_final = df_sku_skst_trans.copy()
merged_df = pd.merge(df_final ,result_df, left_on='STATE', right_on='STATE', how='inner')

In [77]:
merged_df.columns

Index(['SKU', 'DEPT', 'PACKSIZE', 'VENDOR', 'BRAND', 'CLASSID', 'STORE',
       'COST', 'RETAIL', 'CITY', 'STATE', 'ZIP', 'quarter', 'totalunitssold',
       'averageoriginalprice', 'averagesellingprice', 'purchase_count',
       'return_count', 'purchase_prob', 'MenRatio', 'WomenRatio', 'White',
       'Black', 'Native', 'Asian', 'Pacific', 'Income', 'Poverty',
       'Professional', 'Service', 'Office', 'Construction', 'Production',
       'Unemployment'],
      dtype='object')

In [78]:
#define profit margin column
merged_df['PROF_MARG']  = merged_df['COST'] / merged_df['RETAIL'] 
merged_df['DISCOUNTED']  = merged_df['averagesellingprice'] < merged_df['averageoriginalprice']
merged_df.head()

,SKU,DEPT,PACKSIZE,VENDOR,BRAND,CLASSID,STORE,COST,RETAIL,CITY,...,Income,Poverty,Professional,Service,Office,Construction,Production,Unemployment,PROF_MARG,DISCOUNTED
0,3,6505,1,5119207,TURNBURY,113,2007,123.36,440.00,SAN ANTONIO,...,49894.338583,16.335039,29.395669,18.382677,21.708268,15.865354,14.653543,6.028346,0.280364,False
1,156,8101,1,814935,FU DA IN,302,1107,7.50,19.99,LEWISVILLE,...,49894.338583,16.335039,29.395669,18.382677,21.708268,15.865354,14.653543,6.028346,0.375188,False
2,156,8101,1,814935,FU DA IN,302,1207,7.50,19.99,MESQUITE,...,49894.338583,16.335039,29.395669,18.382677,21.708268,15.865354,14.653543,6.028346,0.375188,False
3,156,8101,1,814935,FU DA IN,302,2407,7.50,19.99,SAN ANTONIO,...,49894.338583,16.335039,29.395669,18.382677,21.708268,15.865354,14.653543,6.028346,0.375188,False
4,156,8101,1,814935,FU DA IN,302,4207,7.50,19.99,WICHITA FALLS,...,49894.338583,16.335039,29.395669,18.382677,21.708268,15.865354,14.653543,6.028346,0.375188,False


In [79]:
df_itemcount = df_skstinfo.groupby('STORE')['SKU'].count().reset_index().rename(columns={"SKU": "COUNT_ITEMS"})
item_count_dict = { row['STORE'] : row['COUNT_ITEMS']  for idx,row in df_itemcount.iterrows()}

In [80]:
merged_df['COUNT_ITEMS'] = merged_df['STORE'].apply(lambda x : item_count_dict[x])
merged_df.head()

,SKU,DEPT,PACKSIZE,VENDOR,BRAND,CLASSID,STORE,COST,RETAIL,CITY,...,Poverty,Professional,Service,Office,Construction,Production,Unemployment,PROF_MARG,DISCOUNTED,COUNT_ITEMS
0,3,6505,1,5119207,TURNBURY,113,2007,123.36,440.00,SAN ANTONIO,...,16.335039,29.395669,18.382677,21.708268,15.865354,14.653543,6.028346,0.280364,False,81066
1,156,8101,1,814935,FU DA IN,302,1107,7.50,19.99,LEWISVILLE,...,16.335039,29.395669,18.382677,21.708268,15.865354,14.653543,6.028346,0.375188,False,64308
2,156,8101,1,814935,FU DA IN,302,1207,7.50,19.99,MESQUITE,...,16.335039,29.395669,18.382677,21.708268,15.865354,14.653543,6.028346,0.375188,False,65348
3,156,8101,1,814935,FU DA IN,302,2407,7.50,19.99,SAN ANTONIO,...,16.335039,29.395669,18.382677,21.708268,15.865354,14.653543,6.028346,0.375188,False,52743
4,156,8101,1,814935,FU DA IN,302,4207,7.50,19.99,WICHITA FALLS,...,16.335039,29.395669,18.382677,21.708268,15.865354,14.653543,6.028346,0.375188,False,53911


In [81]:
merged_df = merged_df.drop(columns = ['Pacific', 'Black', 'Native', 'Service', 'Office', 'Production'])

In [85]:
merged_df.drop('WomenRatio',axis=1, inplace=True)

In [86]:
merged_df.head()

,SKU,DEPT,PACKSIZE,VENDOR,BRAND,CLASSID,STORE,COST,RETAIL,CITY,...,White,Asian,Income,Poverty,Professional,Construction,Unemployment,PROF_MARG,DISCOUNTED,COUNT_ITEMS
0,3,6505,1,5119207,TURNBURY,113,2007,123.36,440.00,SAN ANTONIO,...,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,0.280364,False,81066
1,156,8101,1,814935,FU DA IN,302,1107,7.50,19.99,LEWISVILLE,...,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,0.375188,False,64308
2,156,8101,1,814935,FU DA IN,302,1207,7.50,19.99,MESQUITE,...,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,0.375188,False,65348
3,156,8101,1,814935,FU DA IN,302,2407,7.50,19.99,SAN ANTONIO,...,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,0.375188,False,52743
4,156,8101,1,814935,FU DA IN,302,4207,7.50,19.99,WICHITA FALLS,...,56.557874,0.980709,49894.338583,16.335039,29.395669,15.865354,6.028346,0.375188,False,53911


In [ ]:
merged_df.to_csv(os.path.join(DATA_DIR, 'df_final_sku_store_quarter_v3.csv'), index=False)